In [83]:
import pandas as pd
import os
import json

## Read a single JSON

Define a function called read_json. Given a string representing a file path to a json file, this function should open said file and convert its contens into a json object.

In [86]:
# implement me
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

Use the read_json function defined above to read the contents of one of the locations_4.json file acquired in the Data Acquisition Lab.

In [87]:
#file_path = os.path.join('./', 'data', 'locations', 'locations_4.json')
json_contents = read_json('data/locations/locations_4.json')

Display the value of the json_contents variable defined above.

In [89]:
type(json_contents['results'])

list

## Convert JSON Into a DataFrame

Using the json_contents variable, create a Pandas dataframe.

In [91]:
# Create DataFrame here.
df = pd.DataFrame(json_contents['results'])

Display the contents of the DataFrame. 

In [92]:
# Display contents of the DataFrame here.
df.head()

,mindate,maxdate,name,datacoverage,id
0,1893-01-01,2024-12-05,"Lee County, IL",1.0,FIPS:17103
1,1896-05-01,2024-12-05,"Livingston County, IL",1.0,FIPS:17105
2,1906-01-01,2024-12-05,"Logan County, IL",1.0,FIPS:17107
3,1902-08-01,2024-12-05,"McDonough County, IL",1.0,FIPS:17109
4,1893-01-01,2024-12-05,"McHenry County, IL",1.0,FIPS:17111


How many records are in the DataFrame? How many columns does each record have? 

Filter the contents of the DataFrame to only show records where the name column contains the string "Durham".

In [123]:
#df.loc[df.name == 'McDonough County, IL']
filtered_df = df[df['name'].str.contains(r'\bDurham\b', case=False, na=False)]
print(filtered_df)
#there is no place containing Durham

Empty DataFrame
Columns: [mindate, maxdate, name, datacoverage, id]
Index: []


## Read Multiple JSON Files

Define a function called read_all_json_files. Given a string representing a path to a directory, this function should read all of the json files and return a Pandas DataFrame containing all of the objects. In addition to the data from the files, this DataFrame should also contain an additional column called "source". The source column should be populated with the name of the file from which this record originated from.

In [127]:
# implement me
def read_all_json_files(JSON_ROOT):
    all_data = []  # List to store data from all JSON files
    
    # Loop through all files in the directory
    for filename in os.listdir(JSON_ROOT):
        # Check if the file is a JSON file
        if filename.endswith('.json'):
            file_path = os.path.join(JSON_ROOT, filename)
            
            # Read the JSON data
            with open(file_path, 'r') as file:
                data = json.load(file)
                data = data['results']
                
            # Add a 'source' column with the file name
            if isinstance(data, list):
                for record in data:
                    record['source'] = filename  # Add the source file name to each record
                all_data.extend(data)  # Add records from the current file to the list
            else:
                data['source'] = filename  # Add the source file name if it's a single object
                all_data.append(data)
    
    # Convert the list of data into a Pandas DataFrame
    df = pd.DataFrame(all_data)
    
    return df

Use the read_all_json_files function above to read the contents of all locations file acquired in the the Data Acquisition Lab.  
Hint: It is easier if you put all of those files in a dedicated subdirectory ex: (./data/locations/)

In [128]:
df = read_all_json_files('./data/locations')

What is the shape of the DataFrame? Does this match the number of columns and records you expected?

In [132]:
# Validate the shape of the DataFrame here
df.shape

(38862, 6)

Show the first few records of the DataFrame.

In [133]:
# Show the first few records
df.head()

,mindate,maxdate,name,datacoverage,id,source
0,1994-12-15,2024-12-05,"Boston, MA 02118",0.950,ZIP:02118,locations_10.json
1,1994-12-15,2024-12-05,"Roxbury, MA 02119",0.950,ZIP:02119,locations_10.json
2,1994-12-15,2024-12-05,"Roxbury Crossing, MA 02120",0.950,ZIP:02120,locations_10.json
3,1994-12-15,2024-12-05,"Dorchester, MA 02121",0.950,ZIP:02121,locations_10.json
4,1994-12-15,2024-12-05,"Dorchester, MA 02122",0.998,ZIP:02122,locations_10.json


Show the last few records of the DataFrame.

In [135]:
# Show the last few records
df.tail()

,mindate,maxdate,name,datacoverage,id,source
38857,1925-07-01,2024-12-05,"Fombell, PA 16123",1.00,ZIP:16123,locations_13.json
38858,1995-01-19,2024-12-05,"Fredonia, PA 16124",0.95,ZIP:16124,locations_13.json
38859,1894-01-01,2024-12-05,"Greenville, PA 16125",1.00,ZIP:16125,locations_13.json
38860,1907-01-01,2024-12-05,"Grove City, PA 16127",1.00,ZIP:16127,locations_13.json
38861,1995-01-19,2024-12-05,"Hadley, PA 16130",0.95,ZIP:16130,locations_13.json


## Validate Records Are Unique

Provide two different ways to determine there are no duplicate records.  
Hint: The id field should be unique.

In [141]:
# one way to determine there are not duplicate records
df['id'].duplicated().sum()

np.int64(0)

In [142]:
# another way to determine there are no duplicate records
df.duplicated().sum()

np.int64(0)

# Peking In

Show the record with id "CITY:TU000041". Which file did it come from? 

In [144]:
# 
df.loc[df['id']=="CITY:TU000041"]['source']

4001    locations_2.json
Name: source, dtype: object

Show the record with id "CLIM:0405". Which file did it come from? 

In [147]:
# 
df.loc[df['id']=="CLIM:0405"]['source']

3001    locations_3.json
Name: source, dtype: object

How many unique values are there for the source column in the DataFrame?

In [151]:
# 
df['source'].nunique()

39

How many records did the locations_38.json file contain?

In [157]:
# 
df.loc[df['source']=='locations_38.json'].shape[0]

1000

We will work with this DataFrame in a subsequent part of the lab. Instead of repeating all of this work again, let's export the DataFrame to a pickled file.

In [165]:
# export DataFrame to a pickled file called locations_data_frame.pickle. Save it to the data directory.
df.to_pickle('data/locations_data_frame.pickle')